In [ ]:
import geopandas as gpd
import pandas as pd

## Read from Google Sheets

In [ ]:
def load_sheet(
    refresh=False,
    sheet_id="12I5GR8v8H1LVXtjYX5S-v6zuO0up_aY3D3BqplzSu4s",
    sheet_num="0",
    save_path="data/olivia_cities.csv",
):
    """
    Load data from Google Sheet and save again,
    or just load from saved csv
    """
    
    if refresh:
        url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?gid={sheet_num}&format=csv"
        df = pd.read_csv(url)
        df.to_csv(save_path, index=False)
        
    # get data
    return pd.read_csv(save_path)

In [ ]:
# df = load_sheet(refresh=True)

## Cities

In [ ]:
# pd.set_option("display.max.columns", None)

In [ ]:
# get olivia data
df = load_sheet(refresh=False)

# prepare it
df = df.reset_index()
df["date"] = pd.to_datetime(df["date"])

In [ ]:
fp = "data/ne_10m_populated_places.zip"

# get cities data
gdf_full = gpd.read_file(fp)

# prepare it
# gdf = gdf_full.reset_index().loc[:, ["index", "name", "adm1name", "adm0name"]]
gdf = gdf_full

In [ ]:
# gdf_full[gdf_full["NAME"] == "Helsinki"]

### Cleanup

In [ ]:
def keep_first(df, col):
    """Remove duplicate regions, keep first by date"""
    
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")
    return df[[col, "date"]].drop_duplicates(col, keep='first')

### Countries (admin0)

In [ ]:
def merge_countries(countries, visited):
    countries_small = countries["ADMIN"].reset_index()
    merged = pd.merge(countries_small, visited, left_on="ADMIN", right_on="adm0name")
    return countries.iloc[merged['index']] 

In [ ]:
df = load_sheet(refresh=False)
df = keep_first(df, "adm0name")

countries = gpd.read_file("data/ne_10m_admin_0_countries.zip")

# merge together
countries_filtered = merge_countries(countries, df)

In [ ]:
# m = countries_filtered.explore()
# m

### Region (admin1)

In [ ]:
regions = gpd.read_file("data/ne_10m_admin_1_states_provinces.zip")

In [ ]:
# export

# readable_list = ["adm1_code", "iso_a2", "name", "name_alt", "region", "type", "type_en", "postal", "adm0_a3", "iso_3166_2"]
# regions[readable_list].to_csv("admin1.csv")

In [ ]:
def merge_regions(regions, visited):
    region_small = regions["name"].reset_index()
    merged = pd.merge(region_small, visited, left_on="name", right_on="adm1name", how="inner")
    return regions.iloc[merged['index']] 

In [ ]:
df = load_sheet(refresh=False)
df = keep_first(df, "adm1name")

regions = gpd.read_file("data/ne_10m_admin_1_states_provinces.zip")

In [ ]:
merged_regions = merge_regions(regions, df)
merged_regions.shape

In [ ]:
# fin = regions[regions["adm0_a3"] == "FIN"]

In [ ]:
# fin[fin["name"] == "Uusimaa"]

In [ ]:
# merged_regions.explore(
#     # m=m,
#     tiles="CartoDB positron",
#     color='red'
# )

In [ ]:
countries_filtered.explore(
    # column="ADMIN",
    tooltip="ADMIN",
    tiles="CartoDB positron"
)

### Merge

In [ ]:
merged = gdf.merge(df, on=["name", "adm1name", "adm0name"])

In [ ]:
# merged

### Backwards

In [ ]:
# filtered = gdf_full.iloc[merged["index_x"]]

In [ ]:
filtered = merged

In [ ]:
filtered = filtered.drop('Date', axis=1)

In [ ]:
# filtered.explore(
#     column="biked?",
#     tooltip="name",
#     cmap="Set2",
#     tiles="CartoDB positron"
# )

### Find missing stuff

In [ ]:
merged = gdf.merge(df, on=["name", "adm0name"], how='right')

In [ ]:
# merged[merged['index_x'].isna()]

In [ ]:
merged = gdf.merge(df, on=["name", "adm0name"])

In [ ]:
merged = gdf.merge(df, on=["name", "adm1name", "adm0name"])